<a href="https://colab.research.google.com/github/whyeon92/ESAA_YB/blob/mini_project/miniproject3_%EC%A0%84%EC%B2%98%EB%A6%AC%EC%88%98%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#파일 경로 지정 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/ESAAData/miniproject3_data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv(path + "movies_train.csv")
test = pd.read_csv(path + "movies_test.csv")

In [ ]:
# distributor 중복 배급사 전처리
def distr_preprocessing(x) :
    if 'CGV' in x or '무비꼴라쥬' in x :
        return 'CGV'
    elif 'E&M' in x or 'CJ엔터테인먼트' in x or 'CJ 엔터테인먼트' in x:
        return 'CJ E&M'
    elif '롯데' in x :
        return '롯데엔터테인먼트'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x or 'KT' in x: ## 위지윅스튜디오
        return '싸이더스'
    elif '마운틴픽처스' in x or '마운틴픽쳐스' in x :
        return '마운틴픽쳐스'
    elif '메가박스' in x or '씨너스' in x:
        return '메가박스플러스엠'
    elif '쇼박스' in x or '(주)쇼박스' in x : ## 오리온 그룹
        return '쇼박스'
    elif '조이앤시네마' in x or '조이앤컨텐츠그룹' in x or '(주) 케이알씨지' in x or '드림팩트' in x:
        return '조이앤'
    elif '타임스토리' in x :
        return '타임스토리'
    elif '프리비젼' in x :
        return '프리비젼'
    elif '리틀빅픽' in x :
        return '리틀빅픽쳐스'
    elif '콘텐츠판다' in x or '(주)NEW' in x:
        return 'NEW'
    elif 'SK' in x:
        return 'SK'
    elif '마인스' in x:
        return '마인스'
    elif '에이원' in x:
        return '에이원'
    else :
        return x

In [ ]:
full_data =  pd.concat([train, test])
full_data['dir_prev_bfnum'].fillna(0, inplace = True) #결측치 처리
full_data['release_time']=pd.to_datetime(full_data['release_time'])
#release_time 년,월별 데이터 만들기
full_data['release_year'] = full_data['release_time'].dt.year
full_data['release_month'] = full_data['release_time'].dt.month
#불필요한 피처 제거
full_data = full_data.drop(['title','director','release_time'], axis = 1)

full_data['distributor']=full_data['distributor'].apply(distr_preprocessing) #겹치는 배급사 해결
total_dist = full_data.distributor.value_counts()[:14]

def distributor_etc(x) :
  if x in total_dist:
    return x
  else:
    return("기타")

full_data['distributor'] = full_data['distributor'].apply(distributor_etc).apply(lambda x : distributor_etc(x))  #전체 데이터 기준으로 삼아서 배급사 칼럼 기타 레벨 처리
full_data = pd.get_dummies(full_data, columns = ['genre','distributor','screening_rat','release_year','release_month'],drop_first=True, dtype = int) #원핫 인코딩 처리
train = full_data[:600] #데이터 나눔
test = full_data[600:]
test = test.drop('box_off_num', axis = 1)  #train test 합치면서 생긴 test의 y값(NaN) 제거
X_train = train.drop('box_off_num', axis = 1) #train 피처
y_train = train['box_off_num'] #train y
y_train_log = np.log1p(train['box_off_num']) #train y log변환

In [ ]:
print(X_train.shape)
print(test.shape)

(600, 49)
(243, 49)


In [ ]:
full_data.columns

Index(['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor',
       'box_off_num', 'genre_공포', 'genre_느와르', 'genre_다큐멘터리', 'genre_드라마',
       'genre_멜로/로맨스', 'genre_뮤지컬', 'genre_미스터리', 'genre_서스펜스', 'genre_애니메이션',
       'genre_액션', 'genre_코미디', 'distributor_(주)키노아이', 'distributor_CGV',
       'distributor_CJ E&M', 'distributor_NEW', 'distributor_골든타이드픽처스',
       'distributor_기타', 'distributor_롯데엔터테인먼트', 'distributor_마운틴픽쳐스',
       'distributor_쇼박스', 'distributor_시네마달', 'distributor_싸이더스',
       'distributor_어뮤즈', 'distributor_인디스토리', 'distributor_조이앤',
       'screening_rat_15세 관람가', 'screening_rat_전체 관람가',
       'screening_rat_청소년 관람불가', 'release_year_2011', 'release_year_2012',
       'release_year_2013', 'release_year_2014', 'release_year_2015',
       'release_month_2', 'release_month_3', 'release_month_4',
       'release_month_5', 'release_month_6', 'release_month_7',
       'release_month_8', 'release_month_9', 'release_month_10',
       'release_month_11', '

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   distributor     600 non-null    object 
 1   genre           600 non-null    object 
 2   time            600 non-null    int64  
 3   screening_rat   600 non-null    object 
 4   dir_prev_bfnum  600 non-null    float64
 5   dir_prev_num    600 non-null    int64  
 6   num_staff       600 non-null    int64  
 7   num_actor       600 non-null    int64  
 8   release_year    600 non-null    int32  
 9   release_month   600 non-null    int32  
dtypes: float64(1), int32(2), int64(4), object(3)
memory usage: 46.9+ KB


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

mm = MinMaxScaler()
X_train_mm = X_train.copy()
X_train_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = mm.fit_transform(X_train_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])
X_test_mm = test.copy()
X_test_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = mm.transform(X_test_mm[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])

ss = StandardScaler()
X_train_ss = X_train.copy()
X_train_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = ss.fit_transform(X_train_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])
X_test_ss = test.copy()
X_test_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']] = ss.transform(X_test_ss[['time', 'dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor']])

In [ ]:
X_train

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,96,0.00,0,91,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,130,1161602.50,2,387,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,123,220775.25,4,343,4,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,101,23894.00,2,20,6,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,108,1.00,1,251,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,111,3833.00,1,510,7,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,127,496061.00,1,286,6,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,99,0.00,0,123,4,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,102,0.00,0,431,4,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
X_train_mm

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,0.377778,0.000000e+00,0.0,0.104718,0.08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.629630,6.594276e-02,0.4,0.445339,0.12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0.577778,1.253314e-02,0.8,0.394707,0.16,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.414815,1.356433e-03,0.4,0.023015,0.24,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.466667,5.676879e-08,0.2,0.288838,0.08,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.488889,2.175948e-04,0.2,0.586881,0.28,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,0.607407,2.816078e-02,0.2,0.329114,0.24,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,0.400000,0.000000e+00,0.0,0.141542,0.16,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,0.422222,0.000000e+00,0.0,0.495972,0.16,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
X_train_ss

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,genre_공포,genre_느와르,genre_다큐멘터리,genre_드라마,genre_멜로/로맨스,...,release_month_3,release_month_4,release_month_5,release_month_6,release_month_7,release_month_8,release_month_9,release_month_10,release_month_11,release_month_12
0,-0.268953,-0.361285,-0.741416,-0.363216,-0.698066,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1.611324,0.526530,0.950027,1.425124,-0.289043,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1.224208,-0.192546,2.641471,1.159290,0.119980,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0.007558,-0.343023,0.950027,-0.792176,0.938026,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.394674,-0.361284,0.104306,0.603454,-0.698066,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.560581,-0.358356,0.104306,2.168252,1.347050,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
596,1.445417,0.017855,0.104306,0.814913,0.938026,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
597,-0.103047,-0.361285,-0.741416,-0.169882,0.119980,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
598,0.062860,-0.361285,-0.741416,1.690959,0.119980,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
